In [26]:
# Install python client reources

!pip3 install --pre -I "weaviate-client==4.*"

  Using cached weaviate_client-4.2b0-py3-none-any.whl.metadata (3.3 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached validators-0.22.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached Authlib-1.2.1-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached pydantic-2.5.0b1-py3-none-any.whl.metadata (174 kB)
  Using cached grpcio-1.59.2-cp310-cp310-macosx_12_0_universal2.whl.metadata (4.0 kB)
  Using cached grpcio_tools-1.59.2-cp310-cp310-macosx_12_0_universal2.whl.metadata (6.2 kB)
  Using cached cryptography-41.0.5-cp37-abi3-macosx_10_12_universal2.whl.metadata (5.2 kB)
  Using cached protobuf-4.25.0-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached setuptools-68.2.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached pydantic_core-2.14.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.5 kB)
  Using cached typing_extensions-4.8.0-py3-none-any.whl.metadata (3.0 kB)


In [2]:
# Gather resources

!pip install weaviate-demo-datasets

In [40]:
# Connect to instance

import weaviate, os, json
import weaviate.classes as wvc

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.environ['OPENAI_API_KEY']  # Replace with your inference API key
    }
)

In [42]:
# Define collections

if (client.collections.exists("JeopardyCategory")):
    client.collections.delete("JeopardyCategory")

if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyCategory",
    description="A Jeopardy! category",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(name="title", data_type=wvc.DataType.TEXT),
    ]
)

In [43]:
# CrossRef Definition

client.collections.create(
    name="JeopardyQuestion",
    description="A Jeopardy! question",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(name="question", data_type=wvc.DataType.TEXT),
        wvc.Property(name="answer", data_type=wvc.DataType.TEXT),
        # highlight-start
        wvc.ReferenceProperty(
            name="hasCategory",
            target_collection="JeopardyCategory"
        )
        # highlight-end
    ]
)

client.collections.list_all()

{'JeopardyCategory': _CollectionConfigSimple(name='JeopardyCategory', description='A Jeopardy! category', properties=[_Property(data_type=<DataType.TEXT: 'text'>, description=None, index_filterable=True, index_searchable=True, name='title', tokenization=<Tokenization.WORD: 'word'>)], vectorizer=<Vectorizer.TEXT2VEC_OPENAI: 'text2vec-openai'>),
 'JeopardyQuestion': _CollectionConfigSimple(name='JeopardyQuestion', description='A Jeopardy! question', properties=[_Property(data_type=<DataType.TEXT: 'text'>, description=None, index_filterable=True, index_searchable=True, name='question', tokenization=<Tokenization.WORD: 'word'>), _Property(data_type=<DataType.TEXT: 'text'>, description=None, index_filterable=True, index_searchable=True, name='answer', tokenization=<Tokenization.WORD: 'word'>), _Property(data_type=_ReferenceDataType(target_collection='JeopardyCategory'), description=None, index_filterable=True, index_searchable=False, name='hasCategory', tokenization=None)], vectorizer=<Vect

In [44]:
# Upload data. Uses old client (Not sure why)

import weaviate_datasets
dataset = weaviate_datasets.JeopardyQuestions1k()  # instantiate dataset

old_client = weaviate.Client("http://localhost:8080")
dataset = weaviate_datasets.JeopardyQuestions10k()  # instantiate dataset
dataset.upload_objects(old_client, 100)

/Users/davecuthbert/src/weaviate-io/.venv/lib/python3.10/site-packages/weaviate/warnings.py:131: DeprecationWarning: Dep006: You are using the `client.batch()` method, which will be removed in the next major release.
            Please instead use the `client.batch.configure()` method to configure your batch and `client.batch` to enter the context manager.
            See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(
10000it [00:27, 363.28it/s]


True

In [46]:
# Test connection and upload

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    limit=2
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "points": 100.0,
  "answer": "Jonah",
  "air_date": "2001-01-10T00:00:00Z",
  "round": "Jeopardy!",
  "question": "This prophet passed the time he spent inside a fish offering up prayers"
}
{
  "points": 400.0,
  "answer": "lay eggs",
  "air_date": "2004-10-18T00:00:00Z",
  "question": "Pythons are oviparous, meaning they do this",
  "round": "Jeopardy!"
}


In [9]:
# Pretty print output

response = jeopardy.query.fetch_objects( limit=1 )

for o in response.objects:
    print(json.dumps(o.properties, indent=2))
    

{
  "points": 100.0,
  "answer": "Jonah",
  "air_date": "2001-01-10T00:00:00Z",
  "round": "Jeopardy!",
  "question": "This prophet passed the time he spent inside a fish offering up prayers"
}


In [34]:
# Basic Hybrid

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(query="food", limit=3),

for o in response[0].objects:
    print(json.dumps(o.properties, indent=2))


{
  "points": 1000.0,
  "answer": "food stores (supermarkets)",
  "air_date": "1986-02-04T00:00:00Z",
  "round": "Double Jeopardy!",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "points": 200.0,
  "answer": "cake",
  "air_date": "2000-10-04T00:00:00Z",
  "round": "Jeopardy!",
  "question": "Devil's food & angel food are types of this dessert"
}
{
  "points": 800.0,
  "answer": "a closer grocer",
  "air_date": "2008-05-08T00:00:00Z",
  "question": "A nearer food merchant",
  "round": "Jeopardy!"
}


In [10]:
# Explain Hybrid

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    alpha=0.5,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))
    print(o.metadata.explain_score, o.metadata.score)
    

{
  "points": 800.0,
  "answer": "a closer grocer",
  "air_date": "2008-05-08T00:00:00Z",
  "question": "A nearer food merchant",
  "round": "Jeopardy!"
}
(bm25)
(hybrid) Document df958a90-c3ad-5fde-9122-cd777c22da6c contributed 0.007936507936507936 to the score
(hybrid) Document df958a90-c3ad-5fde-9122-cd777c22da6c contributed 0.00819672131147541 to the score 0.016133230179548264
{
  "points": 1000.0,
  "answer": "food stores (supermarkets)",
  "air_date": "1986-02-04T00:00:00Z",
  "round": "Double Jeopardy!",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
(bm25)
(hybrid) Document d6e58390-4f1e-524c-898c-e8aea413022e contributed 0.00819672131147541 to the score
(hybrid) Document d6e58390-4f1e-524c-898c-e8aea413022e contributed 0.007575757575757576 to the score 0.01577247865498066
{
  "points": 400.0,
  "air_date": "1990-05-14T00:00:00Z",
  "answer": "Famine",
  "round": "Jeopardy!",
  "question": "From the Latin for \"hunger\", it's a period whe

In [85]:
# Limit 

import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    alpha=0.5,
    limit=3
).objects

response2 = jeopardy.query.hybrid(
    query="food",
    alpha=0.5,
    limit=3
)

for a in response:
    print(json.dumps(a.properties, indent=2))
print()


{
  "points": 800.0,
  "answer": "a closer grocer",
  "air_date": "2008-05-08T00:00:00Z",
  "question": "A nearer food merchant",
  "round": "Jeopardy!"
}
{
  "points": 1000.0,
  "air_date": "1986-02-04T00:00:00Z",
  "answer": "food stores (supermarkets)",
  "round": "Double Jeopardy!",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "points": 400.0,
  "air_date": "1990-05-14T00:00:00Z",
  "answer": "Famine",
  "round": "Jeopardy!",
  "question": "From the Latin for \"hunger\", it's a period when food is extremely scarce"
}



In [11]:
# autocut ==> auto_limit

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
   query="food",
   # highlight-start
   auto_limit=1
   # highlight-end
).objects,

for o in response[0]:
    print(json.dumps(o.properties, indent=2))

{
  "points": 1000.0,
  "answer": "food stores (supermarkets)",
  "air_date": "1986-02-04T00:00:00Z",
  "question": "This type of retail store sells more shampoo & makeup than any other",
  "round": "Double Jeopardy!"
}
{
  "points": 200.0,
  "air_date": "2000-10-04T00:00:00Z",
  "answer": "cake",
  "question": "Devil's food & angel food are types of this dessert",
  "round": "Jeopardy!"
}


In [18]:
# Fusion
import weaviate.classes as wvc

help(wvc.HybridFusion)

# jeopardy = client.collections.get("JeopardyQuestion")
# response = jeopardy.query.hybrid(
#    wvc.HybridFusion,
#    query="food",
#  ).objects

# for o in response[0]:
#     print(json.dumps(o.properties, indent=2))
    
      

Help on class HybridFusion in module weaviate.collections.classes.grpc:

class HybridFusion(builtins.str, weaviate.util.BaseEnum)
 |  HybridFusion(value, names=None, *, module=None, qualname=None, type=None, start=1)
 |  
 |  Define how the query's hybrid fusion operation should be performed.
 |  
 |  Method resolution order:
 |      HybridFusion
 |      builtins.str
 |      weaviate.util.BaseEnum
 |      enum.Enum
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  RANKED = <HybridFusion.RANKED: 'FUSION_TYPE_RANKED'>
 |  
 |  RELATIVE_SCORE = <HybridFusion.RELATIVE_SCORE: 'FUSION_TYPE_RELATIVE_S...
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from enum.Enum:
 |  
 |  name
 |      The name of the Enum member.
 |  
 |  value
 |      The value of the Enum member.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties inherited from enum.EnumMeta

In [39]:
# Fusion
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
   fusion_type=wvc.HybridFusion("relativeScoreFusion"),
   query="food",
 ).objects

for o in response[0]:
    print(json.dumps(o.properties, indent=2))
    
  

ValueError: 'relativeScoreFusion' is not a valid HybridFusion

In [36]:
# Hybrid with properties

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    return_properties=["question"],
    alpha=0.25,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))
  

{
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "question": "A nearer food merchant"
}
{
  "question": "A silvery food fish, or to heat ore in order to extract metal"
}


In [53]:
# Hybrid with weighted properties

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    return_properties=["question^2", "answer"],
    alpha=0.25,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))


WeaviateQueryException: Query call failed with message no such prop with name 'question^2' found in class 'JeopardyQuestion' in the schema. Check your schema files for which properties in this class are available.

In [54]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    return_properties=["question", "answer"], # <==== ^2 doesn't work
    query_properties=["question^2", "answer"],
    alpha=0.25,
    limit=3
)

response.objects[0].

for o in response.objects:
    print(json.dumps(o.properties, indent=2))
    

{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "Smelt",
  "question": "A silvery food fish, or to heat ore in order to extract metal"
}


In [7]:
# Code completion walkthrough

# jeopardy = client.collections.get("JeopardyQuestion")
# response = jeopardy.query.hybrid(
#    fusion_type=wvc.HybridFusion("relativeScoreFusion"),
#    query="food",
#  ).objects

# for o in response[0]:
#     print(json.dumps(o.properties, indent=2))
  

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
  query="food",
  fusion_type=wvc.HybridFusion.RELATIVE_SCORE,
  limit=3
)
 
#print(response)
for o in response.objects:
    print(json.dumps(o.properties, indent=2))
 

{
  "points": 1000.0,
  "answer": "food stores (supermarkets)",
  "air_date": "1986-02-04T00:00:00Z",
  "question": "This type of retail store sells more shampoo & makeup than any other",
  "round": "Double Jeopardy!"
}
{
  "points": 200.0,
  "air_date": "2000-10-04T00:00:00Z",
  "answer": "cake",
  "round": "Jeopardy!",
  "question": "Devil's food & angel food are types of this dessert"
}
{
  "points": 800.0,
  "air_date": "2008-05-08T00:00:00Z",
  "answer": "a closer grocer",
  "round": "Jeopardy!",
  "question": "A nearer food merchant"
}


In [12]:
# Hybrid fusion

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
  query="food",
  fusion_type=wvc.HybridFusion.RELATIVE_SCORE,
  limit=3
)
 
#print(response)
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "points": 1000.0,
  "answer": "food stores (supermarkets)",
  "air_date": "1986-02-04T00:00:00Z",
  "question": "This type of retail store sells more shampoo & makeup than any other",
  "round": "Double Jeopardy!"
}
{
  "points": 200.0,
  "air_date": "2000-10-04T00:00:00Z",
  "answer": "cake",
  "question": "Devil's food & angel food are types of this dessert",
  "round": "Jeopardy!"
}
{
  "points": 800.0,
  "answer": "a closer grocer",
  "air_date": "2008-05-08T00:00:00Z",
  "question": "A nearer food merchant",
  "round": "Jeopardy!"
}


In [17]:
# Hybrid vector 

inputVector = [0.013085687533020973, -0.00777443777769804, 0.005439540836960077, -0.021052561700344086, -0.02164270170032978, -0.006985447835177183, -0.018974246457219124, -0.025260508060455322, -0.0013630924513563514, -0.03597281128168106, 0.027993107214570045, 0.01635710895061493, -0.02948128432035446, 0.009159981273114681, -0.0026780758053064346, 0.001033544773235917, 0.04051431640982628, 0.001919554895721376, 0.024952609091997147, -0.01960287243127823, 0.0001997531799133867, 0.031405650079250336, 0.021142365410923958, -0.007954045198857784, -0.0008338918560184538, -0.0040572043508291245, 7.381747127510607e-05, -0.019051222130656242, 0.004942412953823805, -0.01888444274663925, 0.028121398761868477, 0.004631306976079941, -0.031559597700834274, -0.003143130801618099, 0.01867917738854885, -0.024208521470427513, 0.0056351847015321255, -0.019333461299538612, -0.0012756941141560674, 0.01737060956656933, 0.020128866657614708, -0.007755194325000048, -0.0071329823695123196, -0.007761608809232712, -0.0074986121617257595, 0.00579554820433259, 0.002782312221825123, 0.01349621918052435, 0.003954571671783924, 0.003170392708852887, 0.017678506672382355, 0.008980373851954937, -0.027454284951090813, -0.004377932287752628, 0.013547535054385662, 0.028506271541118622, -0.011225467547774315, -0.003855146234855056, -8.654634439153597e-05, -0.021770991384983063, -0.004724318161606789, 0.037281379103660583, -0.0543954074382782, 0.01912819594144821, 0.009916898794472218, -0.007806510664522648, 0.0035921495873481035, 0.011757874861359596, -0.004980900324881077, -0.014381427317857742, 0.005952704697847366, 0.009839924052357674, -0.02256639674305916, 0.014561034739017487, 0.01888444274663925, 0.0006859562126919627, 0.0024984683841466904, 0.0033355674240738153, -0.007633317727595568, -0.015330781228840351, 0.02297692745923996, -0.02996879070997238, 0.017473241314291954, 0.01232235599309206, 0.019949259236454964, 0.009769363328814507, -0.038307707756757736, 0.0278134997934103, 0.012264625169336796, -0.007062422577291727, -0.013316611759364605, -0.00465055089443922, 0.013188320212066174, 0.008210627362132072, -0.023862136527895927, 0.006998276803642511, -0.005157300271093845, 0.036024127155542374, -0.0211167074739933, -0.0013109742430970073, -0.0014577071415260434, 0.00858267117291689, -0.004637721460312605, -0.0176656786352396, -0.0035793203860521317, -0.004618478007614613, -0.0015691599110141397, -0.018692007288336754, 0.016536716371774673, -0.012476304545998573, -0.03150828182697296, 0.029789183288812637, 0.01989794336259365, -0.03304777666926384, -0.025234850123524666, 0.0034093346912413836, 0.04154064506292343, 0.013188320212066174, 4.1249832065659575e-06, -0.015279464423656464, -0.01488176267594099, 0.016498230397701263, 0.03025102987885475, -0.015574533492326736, 0.033304356038570404, 0.014445573091506958, 0.018114697188138962, -0.011077932082116604, -0.020719004794955254, -0.008069506846368313, 0.0259789377450943, 0.013079272583127022, 0.014920249581336975, -0.005500479135662317, -0.025119388476014137, 0.017062710598111153, -0.0018938967259600759, -0.003059741575270891, -0.005179751198738813, -0.0017062709666788578, 0.02025715820491314, 0.039077453315258026, -0.01520248968154192, 0.012912494130432606, -0.018730493262410164, 0.022669028490781784, 0.014458402059972286, 0.01916668377816677, -0.013406415469944477, 0.0027758977375924587, 0.02151441015303135, -0.025080900639295578, 0.017999235540628433, -0.01716534234583378, 0.014445573091506958, 0.014868932776153088, -0.008326089009642601, 0.021540068089962006, -0.009820680133998394, 0.0033516038674861193, -0.006863571237772703, 0.0016012326814234257, 0.00046625779941678047, -0.014419914223253727, 0.009442221373319626, 0.021976256743073463, 0.01558736339211464, 0.005356151610612869, 0.013104931451380253, -0.0007625299622304738, -0.0013294160598888993, 0.024580566212534904, -0.011308856308460236, 0.03032800555229187, -0.006594160106033087, 0.023041073232889175, -0.0036466731689870358, 0.004679416306316853, -0.01162958424538374, 0.031764864921569824, -0.01007084734737873, -0.004990521818399429, 0.049135472625494, 0.016062039881944656, -0.005660842638462782, -0.019217999652028084, 0.0008852082537487149, 0.011879751458764076, -0.00190031121019274, -0.03250895440578461, 0.006546051241457462, 0.04238736256957054, 0.014009383507072926, -0.012232552282512188, -0.6835347414016724, -0.006443418096750975, 0.003932120744138956, -0.02776218391954899, 0.027839159592986107, 0.035998471081256866, 0.0012444232124835253, 0.0010808521183207631, 0.005522930063307285, -0.01676764152944088, -0.012976639904081821, 0.010468550026416779, -0.003197654616087675, 0.007274102885276079, -0.00998745858669281, -0.025119388476014137, 0.030302347615361214, -0.01453537680208683, -0.04323408380150795, 0.023002585396170616, -0.02207889035344124, 0.02712072804570198, -0.007633317727595568, -0.0038294880650937557, 0.025991767644882202, -0.002905792323872447, 0.004855816252529621, -0.019949259236454964, 0.00048149237409234047, -0.006250981707125902, -0.017691336572170258, -0.005445955321192741, 0.0037910006940364838, -0.01242498867213726, 0.0669679269194603, -0.00747936824336648, -0.015574533492326736, 0.027505602687597275, 0.024336813017725945, 0.0389748215675354, -0.006587745621800423, -0.012655912898480892, 0.022014744579792023, 0.017524557188153267, -0.009442221373319626, -0.015536046586930752, 0.038102444261312485, 0.016613692045211792, 0.021873624995350838, -0.013098516501486301, 0.025080900639295578, 0.019102538004517555, -0.01688310317695141, 0.014137674123048782, 0.012752130627632141, -0.004724318161606789, 0.031610917299985886, -0.024349642917513847, 0.02603025548160076, 0.010269698686897755, -0.007941216230392456, 0.00834533292800188, 0.0005380206275731325, -0.03150828182697296, 0.0002830421435646713, -0.021629871800541878, -0.012681570835411549, 0.01114207785576582, 0.030995119363069534, 0.0018008856568485498, -0.0017800383502617478, 0.021950598806142807, -0.003095021704211831, -0.00533690769225359, 0.018935760483145714, 0.015279464423656464, 0.019384779036045074, 0.009589755907654762, 0.005211824085563421, 0.0006318334490060806, -0.010859837755560875, 0.00035219904384575784, -0.030481955036520958, 0.0026139302644878626, 0.006504356395453215, 0.008229871280491352, -0.03661426529288292, 0.005215031560510397, 0.0012764959828928113, 0.013175491243600845, 0.02907075360417366, 0.0025481809861958027, -0.004807707387953997, -0.012739301659166813, 0.0014464816777035594, -0.0017463619587942958, -0.03435634449124336, 0.004156630020588636, -0.0032634036615490913, -0.016818957403302193, -0.018127525225281715, 0.015061370097100735, 0.00990406982600689, -0.0008860100642777979, 0.01276495959609747, 0.0259789377450943, 0.01045572105795145, 0.034407660365104675, 0.02747994288802147, -0.042566969990730286, 0.014407085254788399, -0.00013610879250336438, -0.013970895670354366, -0.01984662562608719, 0.003755720565095544, -0.020680518820881844, 0.01708836853504181, -0.010648157447576523, 0.005221446044743061, -0.035177405923604965, -0.0045286742970347404, -0.01708836853504181, 0.013034370727837086, -0.0012845142045989633, -0.0063087125308811665, 0.03338133171200752, 0.012290283106267452, -0.01463800948113203, -0.021296314895153046, -0.010789277963340282, 0.015766970813274384, 0.010949641466140747, 0.011308856308460236, -0.019833797588944435, 0.012848349288105965, 0.007152226287871599, 0.020205840468406677, -0.016331451013684273, 0.013855434022843838, -0.01640842668712139, -0.008107994683086872, -0.010872666724026203, -0.007242029998451471, -0.006908473093062639, 0.004240019246935844, -0.025119388476014137, 0.00765256118029356, 0.0022595261689275503, -0.010179894976317883, -0.009474294260144234, 0.002902585081756115, -0.00389684084802866, -0.021873624995350838, -0.0032698181457817554, 0.023130876943469048, -0.019577214494347572, -0.004086070228368044, -0.006052130367606878, 0.009134323336184025, -0.015125514939427376, -0.0040604118257761, 0.020808808505535126, -0.010808520950376987, 0.016652178019285202, -0.021206511184573174, -0.014625180512666702, 0.002657228382304311, -0.002690904773771763, 0.007165055256336927, -0.03312474861741066, 0.024208521470427513, -0.013060029596090317, 0.010077262297272682, 0.019012734293937683, -0.014304452575743198, 0.033791862428188324, -0.02046242356300354, 0.009673145599663258, 0.019679848104715347, 0.004188702907413244, 0.0067865969613194466, -0.028480613604187965, -0.029686549678444862, -0.0015900072176009417, 0.013791288249194622, -0.0070880805142223835, 0.03576754406094551, 0.022964099422097206, -0.026402298361063004, 0.02801876701414585, -0.022348301485180855, -0.000972606532741338, 0.0023493298795074224, 0.0028400432784110308, 0.004807707387953997, -0.0026235519908368587, 0.02513221837580204, 0.02727467752993107, 0.007575586903840303, -0.018178842961788177, 0.015818286687135696, -0.021142365410923958, 0.018538057804107666, -0.02119368128478527, 0.027428627014160156, -0.016626520082354546, -0.004506223369389772, -0.014445573091506958, 0.018704835325479507, 0.020411105826497078, 0.035536620765924454, -0.026761513203382492, -0.032072763890028, 0.005173336714506149, 0.004531881306320429, 0.036793872714042664, 3.102037589997053e-05, 0.0017367401160299778, 0.006250981707125902, 5.0188864406663924e-05, 0.023733844980597496, -0.015497559681534767, -0.0013462542556226254, -0.008255529217422009, -0.039077453315258026, 0.022438105195760727, -0.013380756601691246, 0.010487793944776058, 0.002185758901759982, -0.026864146813750267, -0.0071714697405695915, 0.02582498826086521, -0.006501149386167526, 0.0024663954973220825, 0.02524768002331257, -0.0340997613966465, -0.005564624443650246, -0.022835807874798775, 0.008512111380696297, 0.005567831918597221, -0.007004691753536463, 0.028711538761854172, 0.022104548290371895, 0.014253135770559311, 0.012604596093297005, -0.01599789410829544, 0.027454284951090813, 0.003919291775673628, -0.026735855266451836, 0.00616117799654603, 0.0073767355643212795, 0.0033099092543125153, -0.027556918561458588, 0.006097032222896814, 0.020244328305125237, 0.006318334490060806, 0.008993202820420265, -0.0174604132771492, 0.014676496386528015, 0.017075538635253906, 0.017203830182552338, 0.007190713658928871, 0.015895262360572815, 0.0021151986438781023, 0.005086740478873253, -0.0014793562004342675, -0.013675826601684093, -0.024939781054854393, -0.019949259236454964, -0.0022739588748663664, 0.0009325155988335609, -0.026043083518743515, 0.00585648650303483, 0.007030349690467119, -0.0026059120427817106, -0.010385161265730858, -0.015689995139837265, 0.01193748228251934, -0.0043073720298707485, -0.01148846372961998, -0.033920153975486755, -0.0259789377450943, 0.02991747297346592, 0.011841264553368092, -0.006587745621800423, 0.0064370036125183105, -0.01544624287635088, 0.022361131384968758, -0.011090761981904507, -0.008403063751757145, 0.02554274909198284, 0.014445573091506958, 0.0013510651187971234, 0.010301771573722363, -0.0035504549741744995, 0.026761513203382492, 0.003954571671783924, -0.02329765446484089, 0.014137674123048782, 0.005426711402833462, 0.012245381250977516, -0.025966109707951546, -0.01708836853504181, -0.008871326223015785, 0.03145696595311165, 0.009756534360349178, -0.007011106237769127, 0.00013781266170553863, -0.00096539017977193, 0.005487649701535702, 0.003014839719980955, 0.005817999131977558, -0.0025032791309058666, 0.004454907029867172, 0.030225371941924095, -0.012816276401281357, -0.015330781228840351, -0.004721110686659813, 0.009102250449359417, 0.0011610339861363173, -0.013714313507080078, -0.05208616703748703, -0.04315711185336113, 0.01729363389313221, 0.05216314271092415, 0.01325246598571539, 0.016947248950600624, 0.008524940349161625, 0.0024663954973220825, -0.024182863533496857, -0.03697348013520241, 0.003208880079910159, 8.890168828656897e-05, -0.021129537373781204, 0.012534036301076412, -0.009121494367718697, 0.01232235599309206, -0.005317664239555597, 0.012976639904081821, -0.019949259236454964, 0.012957395985722542, -0.017588702961802483, 0.027916133403778076, -0.019102538004517555, 0.014920249581336975, 0.012636668980121613, 0.02127065695822239, 0.019551556557416916, 0.0340997613966465, 0.02363121137022972, 0.04351632669568062, 0.025504261255264282, 0.010000287555158138, -0.02642795629799366, -0.011828435584902763, 0.006218908820301294, -0.0006951771210879087, 0.0045126378536224365, -0.02034696191549301, -0.006462662015110254, 0.02955825813114643, 0.006122690625488758, 0.01676764152944088, -0.011828435584902763, 0.005975155625492334, 0.021052561700344086, -0.009467880241572857, -0.01712685637176037, 0.017896601930260658, -0.026966780424118042, -0.017139684408903122, 0.04136103764176369, -0.01558736339211464, -0.023887794464826584, 0.0255812369287014, 0.0028288178145885468, 0.006857156753540039, -0.03340699151158333, 0.021989086642861366, 0.009224127046763897, 0.009467880241572857, -0.0012612614082172513, -0.013027956709265709, -0.017601532861590385, -0.00846720952540636, -0.02196342870593071, -0.008332503959536552, -0.009089421480894089, -0.01669066585600376, -0.013342269696295261, -0.0176528487354517, -0.011475634761154652, -0.022540738806128502, 0.010173480957746506, 0.010847008787095547, -0.012829105369746685, 0.007395979482680559, -0.0013582815881818533, 0.01985945552587509, 0.010616084560751915, -0.0229384396225214, -0.008396649733185768, 0.020116036757826805, 0.022964099422097206, -0.016793299466371536, -0.015600192360579967, 0.002857683226466179, -0.00562235526740551, -0.010096506215631962, 0.010442892089486122, 0.009384490549564362, -0.035741887986660004, 0.020372619852423668, 0.0012748923618346453, -0.016536716371774673, 0.024144375696778297, 0.0035312112886458635, -0.006420967169106007, -0.006241359747946262, -0.007274102885276079, -0.005288798827677965, -0.0012805050937458873, -0.01652388833463192, -0.03604978695511818, -0.014830445870757103, 0.015317952260375023, -0.01439425628632307, -0.007062422577291727, -0.003717233194038272, 0.01530512236058712, 0.0011818812927231193, -0.011007372289896011, 0.0003121080808341503, 0.007594830356538296, 0.019577214494347572, -0.01007084734737873, 0.005054667592048645, 0.022155864164233208, -0.02464471198618412, 0.019782479852437973, -0.008415892720222473, 0.023810818791389465, -0.0044516995549201965, -0.013483390212059021, 0.0035344185307621956, -0.02383647859096527, 0.008184969425201416, -0.0033580183517187834, 0.012950981967151165, -0.0018522021127864718, 0.0011353758163750172, 0.0035023458767682314, -0.006869985722005367, -0.010686644352972507, -0.010288942605257034, 0.02209172025322914, -0.0031655817292630672, -0.02996879070997238, -0.008184969425201416, -0.016049209982156754, 0.01286759227514267, 0.006882815156131983, -0.01534361019730568, -0.02663322351872921, -0.0021617042366415262, -0.008178554475307465, 3.533015478751622e-05, 0.0031046434305608273, -0.0066198185086250305, -0.02991747297346592, 0.0032185018062591553, -0.0324832946062088, 0.004830158315598965, 0.03543398901820183, -0.006206079851835966, -0.014817616902291775, -0.016049209982156754, 0.006026472430676222, -0.016331451013684273, -0.03153394162654877, -0.026607563719153404, -0.015882432460784912, -0.005872522946447134, 0.042361706495285034, 0.023246338590979576, 0.0025674246717244387, 0.023438775911927223, -0.013829775154590607, -0.022874295711517334, 0.006241359747946262, -0.004833365324884653, 0.005878937430679798, -0.013662997633218765, 0.018845954909920692, 0.045645955950021744, 0.015933748334646225, -0.018050551414489746, -0.0034510293044149876, -0.010115749202668667, 0.031200384721159935, -0.009166396223008633, -0.022296985611319542, -0.01138583105057478, -0.022399617359042168, 0.022258497774600983, 0.00609061773866415, -0.02034696191549301, 0.025029584765434265, 0.04243868216872215, -0.013290952891111374, 0.01660086214542389, -0.009269028902053833, 0.018499569967389107, 0.016780469566583633, 0.009467880241572857, -0.023477263748645782, 0.01216199155896902, -0.022887123748660088, -0.0064754909835755825, -0.014868932776153088, -0.012918909080326557, 0.01221972331404686, -0.02399042621254921, -0.0015306726563721895, 0.008993202820420265, 0.009262613952159882, -0.006145141553133726, -0.02159138396382332, -0.01359885185956955, 0.010808520950376987, -0.0158311165869236, 0.0031944471411406994, 0.005423504393547773, -0.010641742497682571, -0.019230829551815987, -0.01965419016778469, -0.020808808505535126, -0.022553566843271255, -0.00031290989136323333, 0.008685304783284664, -0.03953930363059044, 0.02529899589717388, -0.02062920108437538, -0.027736525982618332, 0.007447295822203159, -0.0034991384018212557, 0.02927601896226406, -0.002690904773771763, 0.022630542516708374, -0.0075820013880729675, 0.010628913529217243, -0.03130301833152771, 0.0015370871406048536, -0.0008515318622812629, 0.004493393935263157, 0.0011746649397537112, 0.006767353042960167, 0.030789852142333984, -0.022669028490781784, 0.007966874167323112, 0.017922259867191315, -0.012258210219442844, -0.028121398761868477, 0.020616373047232628, 0.0074152229353785515, 0.017486071214079857, -0.026530589908361435, -0.022887123748660088, -0.015779798850417137, 0.005946290213614702, -0.010134993121027946, 0.015600192360579967, -0.008370990864932537, -0.003912877291440964, -0.01920517161488533, 0.030174056068062782, -0.02866022102534771, 0.016960076987743378, -0.0009373265202157199, -0.011693730019032955, 0.0024744137190282345, -0.03438200429081917, -0.036845192313194275, 0.013778459280729294, 0.010423648171126842, 0.03720440715551376, -0.009435807354748249, 0.018730493262410164, 0.0051861656829714775, 0.014663667418062687, -0.014779129065573215, -0.003261800156906247, 0.012931738048791885, 0.020321302115917206, -0.02894246205687523, -0.03982154279947281, -0.013027956709265709, -0.012348013930022717, 0.008787937462329865, -0.03263724595308304, -0.016434084624052048, 0.023605553433299065, -0.00012598582543432713, -0.01669066585600376, 0.00453829625621438, 0.006190043408423662, 0.004076448269188404, 0.009346003644168377, 0.004714696202427149, -0.009269028902053833, -0.028121398761868477, 0.00096539017977193, 0.006196457892656326, -0.012091431766748428, -0.017062710598111153, -0.028403639793395996, 0.0158311165869236, -0.00407965574413538, 0.012213308364152908, -0.013650167733430862, 8.679691381985322e-05, 0.00011275580618530512, -0.021706845611333847, 0.02098841592669487, -0.03250895440578461, 0.013509048148989677, 0.004486979451030493, 0.006818669382482767, -0.015189660713076591, 0.004679416306316853, 0.018050551414489746, 0.003131905337795615, -0.005478028208017349, -0.018807468935847282, 0.0045511252246797085, 0.017383437603712082, 0.01033384446054697, -0.0013133796164765954, 0.03284250944852829, 0.007120153401046991, -0.0002808371209539473, -0.0225279089063406, -0.018910100683569908, 0.022681858390569687, -0.007857827469706535, -0.0041854954324662685, -0.008326089009642601, -0.023900622501969337, 0.016472570598125458, 0.01903839223086834, -0.0012067377101629972, 0.005118812900036573, -0.010891910642385483, 0.01415050309151411, -0.016536716371774673, 0.020885784178972244, -0.000979822943918407, -0.00815931148827076, -0.005192580632865429, 0.010693059302866459, -0.015433413907885551, 0.011905409395694733, -0.01737060956656933, -0.0032521781977266073, 0.0033259454648941755, 0.058603353798389435, 0.004692245274782181, 0.008236285299062729, 0.0031463380437344313, -0.019833797588944435, -0.02301541529595852, -0.022309813648462296, -0.0017174964305013418, -0.02359272539615631, -0.010077262297272682, 0.01162958424538374, 0.009878410957753658, -0.013868262991309166, -0.02354140765964985, 0.03019971400499344, -0.015972236171364784, -0.0067032077349722385, -0.015215318650007248, 0.004316993989050388, 0.02472168579697609, 0.006780182011425495, -0.03345830738544464, 0.012277454137802124, 0.01171297300606966, 0.011180565692484379, 0.008447965607047081, 0.004608856048434973, 0.014137674123048782, -0.0196926761418581, 0.018858784809708595, -0.002015773206949234, -0.027659550309181213, -0.02025715820491314, 0.008499282412230968, 0.007973289117217064, 0.025837818160653114, 0.0005063487333245575, -0.013534706085920334, -0.02408022992312908, -0.044722262769937515, -0.01232235599309206, 0.012617425061762333, 0.005747439339756966, 0.0075820013880729675, -0.017396267503499985, -0.014830445870757103, 0.0022771661169826984, 0.00034357947879470885, -0.028249690309166908, -0.005426711402833462, 0.008909814059734344, 0.007049593608826399, 0.021052561700344086, -0.0023284826893359423, -0.0052920058369636536, 0.00941656343638897, -0.026094401255249977, 0.00616117799654603, 0.003858353476971388, -0.01684461534023285, 0.008961129933595657, -0.009673145599663258, -0.011963141150772572, -0.014009383507072926, -0.019153853878378868, -0.01697290688753128, -0.007594830356538296, 0.02712072804570198, -0.00564159918576479, -0.00581479212269187, -0.021527238190174103, 0.006728865671902895, -0.0043073720298707485, 0.020552227273583412, -0.003710818709805608, 0.012918909080326557, 0.015882432460784912, 0.005898181349039078, 0.014432743191719055, -0.019461752846837044, 0.00024976665736176074, 0.0008150491048581898, 0.0021264241077005863, -0.008306846022605896, -0.005381809547543526, 0.010410819202661514, -0.010173480957746506, -0.002830421319231391, 0.019436094909906387, -0.017563045024871826, -0.020770322531461716, 0.004037960898131132, -0.018730493262410164, -0.00880718044936657, 0.011745045892894268, 0.18124960362911224, 0.010731546208262444, -0.0013294160598888993, 0.03794849291443825, 0.010956056416034698, 0.03723006322979927, 0.02935299277305603, 0.005340115167200565, 0.01033384446054697, 0.030558928847312927, -0.001111321267671883, 0.016754811629652977, -0.010641742497682571, 0.0006975826108828187, 0.00545878428965807, -0.01688310317695141, -0.014868932776153088, -0.025183534249663353, -0.009166396223008633, -0.013534706085920334, -0.010507036931812763, -0.019820967689156532, -0.013444902375340462, -0.0038294880650937557, 0.029583917930722237, 0.0178581140935421, -0.017973575741052628, -0.01335509866476059, 0.034895166754722595, 0.019346291199326515, 0.008088750764727592, -0.00846079457551241, 0.0025722356513142586, -0.00622211629524827, -0.02042393572628498, 0.014843274839222431, -0.005320871248841286, -0.018974246457219124, 0.009711632505059242, -0.012822690419852734, -0.007197128143161535, -0.00787707045674324, 0.008358161896467209, -0.03320172429084778, -0.020475251600146294, 0.0036113932728767395, -0.006940545979887247, 0.0025129010900855064, 0.005933461245149374, 0.008499282412230968, -0.019051222130656242, -0.026684539392590523, 0.0005632779211737216, 0.0278134997934103, 0.0119759701192379, 0.00838382076472044, 0.020924270153045654, 0.008916228078305721, 0.0006194052402861416, 0.005465198773890734, 0.003152752760797739, 0.03515174984931946, -0.013239637017250061, 0.029455626383423805, -0.017357779666781425, 0.016472570598125458, -0.01102020125836134, -0.019820967689156532, -0.0022017951123416424, -0.01359885185956955, 0.035741887986660004, -0.022476593032479286, -0.010705888271331787, -0.011110004968941212, -0.009179225191473961, -0.01716534234583378, 0.00860833004117012, 0.02606874145567417, -0.0032970800530165434, 0.019872283563017845, -0.014714984223246574, 0.007575586903840303, -0.01680612750351429, -0.0025658211670815945, -0.01439425628632307, -0.0373583547770977, 0.014843274839222431, -0.007421637419611216, -0.01692158915102482, -0.02561972290277481, -0.0077038779854774475, -0.0021151986438781023, -0.008281187154352665, 0.012809861451387405, 0.01684461534023285, -0.005240689497441053, -0.03835902363061905, 0.013765630312263966, 0.014265964739024639, -0.008075921796262264, -0.01684461534023285, -0.03864126652479172, 0.016010724008083344, 0.0005817197379656136, -0.025324653834104538, -0.016062039881944656, -0.013996553607285023, 0.012232552282512188, 0.008967544883489609, -0.007344662677496672, 0.014342939481139183, 0.004150215536355972, -0.004124557599425316, -0.005811584647744894, -0.006940545979887247, -0.008800766430795193, 0.008653231896460056, -0.010276113636791706, 0.021976256743073463, -0.012745716609060764, 0.03807678446173668, -0.003932120744138956, 0.0017559838015586138, 0.020885784178972244, -0.04903284087777138, -0.02650493197143078, -0.02160421386361122, 0.004836572799831629, 0.015420584939420223, -0.03271421790122986, 0.014330110512673855, -0.024696027860045433, -0.019872283563017845, 0.0034510293044149876, -0.006882815156131983, 0.000971804722212255, 0.006084203254431486, 0.01888444274663925, -0.029378650709986687, 0.001085663097910583, -0.007081666029989719, -0.0034927239175885916, 0.014253135770559311, -0.014330110512673855, -0.004567161668092012, -0.018409766256809235, 8.809987048152834e-05, -0.0016373145626857877, -0.0082234563305974, -0.010045189410448074, -0.0034959311597049236, -0.020359789952635765, -0.010276113636791706, -0.005295213311910629, 0.016087697818875313, -0.040411680936813354, -0.03256027027964592, -0.004791670944541693, -0.02062920108437538, 0.005747439339756966, -0.014009383507072926, -0.014060699380934238, 0.024991096928715706, -0.006174006965011358, -0.007434466388076544, -0.022515080869197845, -0.16339148581027985, 0.029609575867652893, 0.006722451187670231, -0.037332694977521896, 0.021976256743073463, -0.022361131384968758, -0.0043458594009280205, -0.005083533003926277, 0.009403734467923641, 0.0114692198112607, 0.020475251600146294, -0.01242498867213726, -0.04600517079234123, -0.031610917299985886, 0.02942996844649315, -0.007530685048550367, -0.0010920775821432471, 0.019667018204927444, 0.007312590256333351, 0.022194352000951767, 0.013983724638819695, -0.006613404024392366, 0.018358450382947922, -0.01757587492465973, -0.0015042126178741455, 0.021334802731871605, 0.004499808885157108, 0.01993642933666706, -0.014368598349392414, 0.008563428185880184, 0.012418573722243309, 0.0027261849027127028, 0.014766300097107887, 0.03820507600903511, 0.0012227741535753012, -0.01509985700249672, -0.01033384446054697, -0.0005817197379656136, 0.0033291529398411512, 0.016023552045226097, 0.010147822089493275, 0.027736525982618332, -0.002987577812746167, -0.0002890557807404548, -0.01903839223086834, 0.007235615514218807, 0.02907075360417366, -0.006546051241457462, 0.011000958271324635, -0.0002021586406044662, 0.0162416473031044, -0.013393585570156574, 0.0011882958933711052, 0.008043848909437656, 0.020282816141843796, 0.012328770011663437, -0.0011650430969893932, -0.0016252873465418816, 0.005708951968699694, 0.005198995117098093, -0.003941742703318596, 0.0032024653628468513, 0.014458402059972286, -0.00858267117291689, -0.00953844003379345, -0.02184796705842018, -0.02297692745923996, 0.010122164152562618, 0.0176656786352396, 0.005939875729382038, -0.004679416306316853, -0.002729392144829035, 0.0031607707496732473, -0.010494207963347435, 0.009461465291678905, 0.009929727762937546, -0.0269924383610487, -0.00016016335575841367, 0.009711632505059242, -0.01721666008234024, -0.0063087125308811665, -0.008204213343560696, -0.0033355674240738153, 0.012687984853982925, 0.004515845328569412, 0.005776304751634598, -0.009666730649769306, 0.0033387746661901474, -0.007780852261930704, -0.010115749202668667, 0.028506271541118622, -0.0077038779854774475, -0.007492197677493095, -0.02209172025322914, -0.011360173113644123, 0.0118733374401927, 0.012630254030227661, 0.011918239295482635, 0.007973289117217064, -0.009647487662732601, 0.01834562048316002, -0.002833628561347723, -0.019782479852437973, 0.011340929195284843, 0.019089708104729652, 0.014201819896697998, 0.0023236717097461224, 0.010186309926211834, 0.023977598175406456, 0.0018618239555507898, 0.005205409601330757, 0.033637914806604385, 0.013662997633218765, 0.00544916233047843, -0.015510388650000095, 0.03186749666929245, -0.005378602538257837, -0.03235500305891037, 0.013752801343798637, 0.0006743298727087677, -0.004426041152328253, -0.012495548464357853, 0.010256869718432426, 0.008531355299055576, 0.0034317856188863516, -0.0025112973526120186, -0.07861676067113876, -0.007062422577291727, 0.0063311634585261345, 0.02537596970796585, -0.02704375423491001, 0.027864817529916763, 0.004207946360111237, 0.013329440727829933, -0.01579262875020504, 0.01815318502485752, 0.007120153401046991, -0.02359272539615631, -0.02627400867640972, -0.013752801343798637, 0.015459071844816208, 0.029301676899194717, 0.0025161083322018385, -0.003045308869332075, -0.017601532861590385, -0.0025818573776632547, -0.009910483844578266, -0.010410819202661514, -0.011167735792696476, 0.0014865725534036756, -0.009756534360349178, 0.009429392404854298, -0.014278794638812542, 0.007960460148751736, 0.007152226287871599, -0.001069626654498279, 0.018576543778181076, -0.012874007225036621, 0.002876926911994815, -0.04015510156750679, -4.9462214519735426e-05, -0.0053080422803759575, 0.007819339632987976, -0.001986907795071602, 0.012354428879916668, -0.01244423259049654, 0.01266874186694622, 0.0017367401160299778, -0.011693730019032955, -0.047108475118875504, 0.020513739436864853, -0.013855434022843838, -0.007004691753536463, 0.005785926710814238, 0.029455626383423805, -0.03610110282897949, -0.005981570575386286, 0.00186984206084162, -0.013791288249194622, -0.0074537103064358234, 0.014933078549802303, -0.02078315056860447, 0.012315941043198109, -0.025812160223722458, -0.020026233047246933, 0.007883485406637192, -0.011270369403064251, -0.020885784178972244, -0.023772332817316055, 0.0340997613966465, 0.007941216230392456, 0.013470560312271118, -0.025914791971445084, -0.050854574888944626, 0.0158311165869236, -0.01140507496893406, -0.01242498867213726, -0.007203542627394199, -0.014522546902298927, -0.006959789898246527, -0.015407755970954895, -0.01896141842007637, -0.024708857759833336, 0.0023733845446258783, 0.020847296342253685, -0.029045093804597855, -0.010167066007852554, -0.021283484995365143, 0.010391575284302235, -0.018640689551830292, 0.012104260735213757, 0.018012063577771187, 0.012072187848389149, 0.009878410957753658, -0.012392915785312653, -0.01875615119934082, 0.009038104675710201, -0.0017110819462686777, 0.03325304016470909, -0.020654859021306038, -0.02087295427918434, 0.02010320872068405, -0.007235615514218807, -0.01566433720290661, 0.013778459280729294, -0.023823648691177368, -0.013290952891111374, 0.008775108493864536, -0.044593971222639084, 0.0067032077349722385, -0.018089039251208305, -0.015369268134236336, 0.013329440727829933, -0.008563428185880184, -0.00937807559967041, 0.005975155625492334, 0.0062926760874688625, 0.015561704523861408, -0.012803447432816029, 0.009576926939189434, 0.010372331365942955, -0.001885878504253924, 0.0023894209880381823, -0.005628770217299461, 0.02927601896226406, 0.014214648865163326, 0.01672915369272232, -0.001007084734737873, 0.019795309752225876, 0.013675826601684093, -0.0036242222413420677, -0.0113665871322155, 0.003111058147624135, 0.008845668286085129, -0.005073911044746637, 0.028531929478049278, -0.010391575284302235, -0.0194874107837677, 0.019064050167798996, -0.019269315525889397, -0.012578938156366348, 0.032457634806632996, 0.0022819770965725183, -0.02359272539615631, -0.003890426130965352, 0.015266635455191135, -0.0046954527497291565, 0.013079272583127022, -0.0031190761364996433, -0.02655624784529209, 0.019679848104715347, -0.025914791971445084, -0.014817616902291775, -0.014022212475538254, -0.008685304783284664, 0.00998745858669281, -0.005138056818395853, 0.01599789410829544, 0.029096411541104317, 0.011430732905864716, 0.013470560312271118, -0.03779454529285431, 0.0014408689457923174, -0.02139894850552082, 0.010564768686890602, 0.02732599526643753, 0.008787937462329865, 0.001094483071938157, 0.02256639674305916, 0.006472283508628607, -0.025145046412944794, -0.016536716371774673, -0.024991096928715706, -0.010956056416034698, -0.010244040749967098, 0.0061643850058317184, 0.023310484364628792, -0.02789047546684742, -0.022438105195760727, 0.007934801280498505, 0.0033034945372492075, -0.007524270098656416, 0.018935760483145714, 0.003867975203320384, 0.007267688401043415, 0.001003877492621541, -0.0003912877000402659, 0.0442604124546051, 0.019923601299524307, 0.026812830939888954, -0.020885784178972244, 0.03758927807211876, 0.0017784347292035818, -0.0034702729899436235, 0.0004931187140755355, 0.0014713379787281156, 0.01404787041246891, 0.0005945488810539246, 0.016139013692736626, 0.0022739588748663664, 0.0276082344353199, -0.0026524176355451345, 0.020924270153045654, 0.016190331429243088, -0.007299760822206736, -0.027556918561458588, 0.00996821466833353, 0.018076209351420403, 0.019743993878364563, -0.010442892089486122, -0.01708836853504181, -0.013483390212059021, -0.011905409395694733, -0.014060699380934238, -0.04857099428772926, -0.018858784809708595, -0.025606894865632057, 0.0043234084732830524, 0.025812160223722458, 0.010507036931812763, 0.016382766887545586, 0.030071422457695007, -0.01530512236058712, 0.004288128577172756, -0.02590196393430233, -0.002894566860049963, 0.00810157973319292, 0.0013117759954184294, 0.002196984365582466, 0.015292293392121792, 0.009564097970724106, -0.0024022499565035105, -0.007081666029989719, 0.002812781371176243, 0.002102369675412774, -0.03096945956349373, 0.015420584939420223, 0.009589755907654762, -0.0064754909835755825, -0.0073318337090313435, -0.002026998670771718, -0.015394926071166992, -0.02712072804570198, 0.00927544292062521, 0.015035711228847504, 0.020167354494333267, -0.019820967689156532, 0.04213078320026398, 0.01640842668712139, 0.010244040749967098, -0.002860890468582511, 0.021078219637274742, 0.01858937367796898, 0.0047179036773741245, -0.013162662275135517, -0.014894591644406319, -0.013053614646196365, 0.004884681664407253, 0.014407085254788399, 0.007594830356538296, -0.001111321267671883, -0.027582576498389244, -0.0024455483071506023, -0.021655529737472534, -0.007505026645958424, -0.009429392404854298, -0.016177501529455185, 0.0072484444826841354, 0.017139684408903122, 0.014561034739017487, 0.007075251545757055, -0.02289995364844799, 0.006411345209926367, 0.018768981099128723, 0.0026347774546593428, -0.008871326223015785, -0.03989851847290993, 0.008204213343560696, 0.013060029596090317, -0.03299646079540253, -0.015536046586930752, 0.02464471198618412, -0.02894246205687523, -0.006485112942755222, 0.008768693543970585, 0.016831785440444946, 0.035664912313222885, -0.0032938728109002113, 0.022425275295972824, -0.02050090953707695, -0.00025557982735335827, 0.013239637017250061, -0.016652178019285202, 0.004143801052123308, -0.0032072763424366713, -0.025273337960243225]


jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    query_properties=["question^2", "answer"],
    vector=inputVector,
    alpha=0.25,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "points": 200.0,
  "answer": "pasta",
  "air_date": "1984-12-04T00:00:00Z",
  "question": "Type of food that comes in shapes of bow ties, elbows & wagon wheels",
  "round": "Jeopardy!"
}
{
  "points": 800.0,
  "answer": "a closer grocer",
  "air_date": "2008-05-08T00:00:00Z",
  "question": "A nearer food merchant",
  "round": "Jeopardy!"
}
{
  "points": 200.0,
  "answer": "cake",
  "air_date": "2000-10-04T00:00:00Z",
  "question": "Devil's food & angel food are types of this dessert",
  "round": "Jeopardy!"
}


In [35]:
# Hybrid where
from weaviate.classes import Filter

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    filters=Filter("round").equal("Double Jeopardy!"),
    alpha=0.50,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))
    

WeaviateQueryException: Query call failed with message extract params: no such prop with name 'round' found in class 'JeopardyQuestion' in the schema. Check your schema files for which properties in this class are available.

In [48]:
from weaviate.classes import Filter

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    filters=Filter("round").equal("Double Jeopardy!"),
    limit=3
)

# print result objects 
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "points": 2000.0,
  "answer": "Baffin Bay",
  "air_date": "1997-04-04T00:00:00Z",
  "question": "For most of the year, ice floes & icebergs prevent navigation in this bay between Greenland & NE Canada",
  "round": "Double Jeopardy!"
}
{
  "points": 600.0,
  "air_date": "2000-07-17T00:00:00Z",
  "answer": "\"Homeward Bound And Gagged\"",
  "round": "Double Jeopardy!",
  "question": "Simon & Garfunkel hit song about being tied up & unable to speak"
}
{
  "points": 800.0,
  "answer": "garage",
  "air_date": "2002-11-19T00:00:00Z",
  "round": "Double Jeopardy!",
  "question": "This French word originally meant \"a place where one docks\" a boat, not a car"
}


In [47]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects()

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "points": 100.0,
  "answer": "Jonah",
  "air_date": "2001-01-10T00:00:00Z",
  "question": "This prophet passed the time he spent inside a fish offering up prayers",
  "round": "Jeopardy!"
}
{
  "points": 400.0,
  "answer": "lay eggs",
  "air_date": "2004-10-18T00:00:00Z",
  "question": "Pythons are oviparous, meaning they do this",
  "round": "Jeopardy!"
}
{
  "points": 800.0,
  "air_date": "1997-04-09T00:00:00Z",
  "answer": "Wal-Mart",
  "question": "In August 1996 this chain, the world's largest retailer, opened its first outlets in China",
  "round": "Double Jeopardy!"
}
{
  "points": 300.0,
  "air_date": "1995-09-27T00:00:00Z",
  "answer": "the Hubble telescope",
  "round": "Jeopardy!",
  "question": "In 1990 this space telescope provided the first picture of Charon as completely separate from Pluto"
}
{
  "points": 1000.0,
  "answer": "Hank Aaron",
  "air_date": "2002-05-10T00:00:00Z",
  "question": "M.C. Hammer earned his nickname from his resemblance to this \"Hammerin'\" h